---
title: "05-SAS 数据库与数据集"
subtitle: "SAS 数据库与数据集的基本知识"
author: "Simon Zhou"
date: "2025-05-25"
data-modified: today
format: 
    html:
        code-fold: false
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 2
---

In [1]:
import saspy
sas = saspy.SASsession()
%load_ext saspy.sas_magic

Using SAS Config named: winlocal
SAS Connection established. Subprocess id is 6976



## 数据库

### 数据库与库标记

SAS 数据库是一个包含数据集的文件夹。每个数据库都包含一个或多个数据集。SAS 数据库的扩展名为 `.sas7bdat`。SAS 数据库可以存储在本地计算机上，也可以存储在远程服务器上。

SAS 数据库可以通过 SAS 程序或 SAS 界面创建和管理。SAS 数据库可以包含多种类型的数据集，包括 SAS 数据集、Excel 数据集、CSV 数据集等。SAS 数据库可以通过 SAS 程序或 SAS 界面进行访问和管理。

数据库使得 SAS 系统能够在 SAS 程序中调用指定的文件。

为了使用 SAS 数据库，需要为每个数据库制定一个库标记来识别。库标记又称库逻辑名或库关联名。库标记是一个 SAS 程序中的变量名，用于引用数据库中的数据集。库标记可以是任意合法的 SAS 变量名，但通常使用大写字母和下划线来命名。例如，`WORK`、`SASUSER`、`MYDATA` 等都是合法的库标记。

### 数据库类型

SAS 数据库有两种类型：永久数据库和临时数据库。
- **永久数据库**：永久数据库是指在 SAS 程序运行结束后仍然存在的数据库。永久数据库通常存储在本地计算机或远程服务器上。永久数据库可以通过 SAS 程序或 SAS 界面进行访问和管理；在安装 SAS 时，SAS 会自动创建一个永久数据库，名为 `SASUSER`，用于存储用户的个人数据集。永久数据库的库标记通常以大写字母和下划线命名，例如 `MYDATA`。
- **临时数据库**：临时数据库是指在 SAS 程序运行结束后自动删除的数据库。临时数据库通常存储在内存中。临时数据库只能通过 SAS 程序进行访问和管理，不能通过 SAS 界面进行访问和管理；临时数据库一般存储在 `WORK` 目录中，`WORK` 目录是一个临时数据库，用于存储临时数据集。临时数据库的库标记通常以小写字母和下划线命名，例如 `work`、`temp` 等。

永久型数据库的库标记也可以由用户使用 `libname` 语句来自行定义，`libname` 语句的一般形式为：

```sas
libname libref 'path';
```

其中，`libref` 是库标记，`path` 是数据库的路径。`libname` 语句可以在 SAS 程序的任何位置使用，但通常在程序的开头使用。`libname` 语句可以用于创建永久数据库，也可以用于访问已有的永久数据库。

启动 SAS 后，除了 `Sasuser` 数据库外，还会自动生成另外两个永久型数据库，他们的库标记分别为 `Sashelp` 和 `Maps` 。

另外，根据用户安装的模块，在启动 SAS 后，会自动生成一些特殊的永久型数据库。

## 数据集

### 数据集的类型

SAS 数据集有两种类型: SAS 数据文件（SAS data files）和 SAS 数据视窗（SAS data views）。

SAS 的数据文件不仅包括描述部分，也包括数据部分。SAS 的视窗文件只有描述部分，没有数据部分，它的描述部分包括的信息使 SAS 过程可以访问到实际上并不包含在数据视窗内部的数据。

一般情况下所说的 SAS 数据集指的是 SAS 数据文件。

### 变量类型

数据集中的变量可以有两种类型:数值型和字符型。

数值型变量只允许变量值为数字,SAS 过程可以对这些数字进行统计运算,如计算变量的均数、标准差等。一般情况下, SAS 默认数值型变量小数点后保留两位有效数值,而小数点前的位数就是该变量值所有数值中的最大位数。

用户也可以用 `length` 语句或 `attrib` 语句自己定义变量长度。
数值型变量当数据缺失时,SAS 表示为 `.`。

字符型变量允许变量值为中、英文字母、各种符号和数字,此时的数字被当作字符处理,无法进行统计运算。字符型变量的默认长度为8个字
节,然而 SAS 规定字符型变量的最大长度不能超过 200 个字节。字符型变量数据缺失时,SAS表示为空格。

### 数据集的命令

每个 SAS 数据集都有一个两级文件名,第一级是库标记,第二级是文件名,两者之间用 `.` 分隔。在建立 SAS数据集时,可以通过指定两级文
件名定义 SAS 数据集,便于以后用 SAS 过程来识别和处理。

如例 `data prg1-1`,表示该数据集为临时数据集,临时数据集的第一级库标记应为 `Work`,也可以省略该库标记。
如果将该数据集存放在硬盘的另一个目录中如 `Sasuser`,则数据集名为 `sasuser.prg1-1`,其物理位置就是 `Sasuser` 这个库标记所指定的文件夹,该数据集将永久保留在硬盘上。

## 数据集的建立

### 创建 SAS 数据集

建立数据集一般在数据步中完成,可以通过直接录人数据或者导人其他格式的数据文件中的数据,来建立 SAS 可以识别的 SAS 数据集。主要有如下几种方法：

1. `input` 和 `datalines` 语句

In [ ]:
%%SAS
/*程序2-1*/
/*10例肾移植患者的部分数据*/
/*M为男性,F为女性*/
data prg2_1;
	input no sex $ age blood $ surt;
datalines;
1 M 41 A 368
2 M 26 B 745
3 F 35 B 401
4 M 37 AB 552
5 F 37 A 478
6 F 39 0 628
7 M 28 0 549
8 M 31 B 128
9 M 43 AB 463
10 M 29 A 512
;
run;

Using SAS Config named: winlocal
SAS Connection established. Subprocess id is 1952



程序 2-1 中第一行 `data prg2_1;` 是要求建立一个文件名为 `prg2_1` 的数据集,该数据集是一个临时数据集,系统会自动将其存放在 `Work`目录中。第二行 `input` 语句定义了数据集的变量名和变量类型,其中 `@` 符号表示在同一行中输入数据,而 `datalines` 语句则表示数据的输入结束。最后一行 `run;` 表示程序结束。

第一行会建立一个名为 `prg2_1` 的数据集,该数据集是一个临时数据集,系统会自动将其存放在 `Work` 数据库目录中。文件的后缀名为 `sas7bdat`,所以从
Windows资源管理器中查看该文件,文件名为 `prg2_1.sas7bdat`。如果需建立永久型数据集,可在 `prg2_1` 前面加上库标记,如 `sasuser.prg2_1`,则该
数据集将保存 Sasuser 数据库中,退出 SAS 也不会将该数据集删除。

第二行 `input no sex $ age blood $ surt;` 是要求在 `prg2_1.sas7bdat` 数据集中建立5个变量,
它们的变量名分别为 `no`、`sex`、`age`、`blood` 和 `surt`,其中`sex`和`blood`变量名后面加上了一个符号 `$`,表示这些变量为字符型变量,其他未加 `$` 的变量则默认为数值型变量。

第三行 `datalines;` 表明开始对变量进行赋值,它向 SAS 指示下一行开始是数据行,直到分号出现,数据行赋值结束。而该分号必须出现在所有数据的下一行,才表示结束数据行。数据行中不同变量的数据之间可用一个或多个空格
分开。

最后一行 `run;` 表示 `data` 步的结束,当后面还有其他数据步或过程步语句,该语句可省略。

**打印数据集**

In [ ]:
%%SAS
/*程序2-2*/
/*print过程将数据显示在输出窗口中*/
proc print data = prg2_1;
run;

观测,no,sex,age,blood,surt
1,1,M,41,A,368
2,2,M,26,B,745
3,3,F,35,B,401
4,4,M,37,AB,552
5,5,F,37,A,478
6,6,F,39,0,628
7,7,M,28,0,549
8,8,M,31,B,128
9,9,M,43,AB,463
10,10,M,29,A,512


**横向输入**

如果数据集中变量较少，而观测值较多，可以使用横向输入的方法来输入数据，具体方法是在 `input` 语句中的变量名加上两个 `@@` 符号，表示在同一行中输入数据，则在数据行中的数据可以横行排列,每个数据之间用空格分隔。

具体如下程序 2-3 所示：

In [4]:
%%SAS
/*程序2-3*/
/*变量比较少，观测比较多，可以采用横向输入方法；具体方法是在input语句中的变量名后加上两个@@，则在数据行中的数据可以横行排列*/
data prg2_2;
	input test @@;
datalines;
41 26 35 47 37 39 28 31 43 29
;
run;

proc print data = prg2_2;
run;

观测,test
1,41
2,26
3,35
4,47
5,37
6,39
7,28
8,31
9,43
10,29
